In [46]:
from pynq import Overlay
ol = Overlay('matrix.bit')

In [47]:
dma = ol.axi_dma_0

In [48]:
data_send = dma.sendchannel
data_recv = dma.recvchannel

In [49]:
import numpy as np
import random 

In [50]:
data_A = np.random.randint(low=0, high=10, size=(8, 8)).astype(np.float32)
data_B = np.random.randint(low=0, high=10, size=(8, 8)).astype(np.float32)

In [51]:
data_A, data_B

(array([[4., 8., 6., 4., 4., 3., 3., 3.],
        [9., 5., 0., 1., 7., 0., 5., 0.],
        [2., 0., 7., 0., 2., 3., 8., 5.],
        [6., 8., 5., 7., 4., 0., 3., 1.],
        [0., 9., 0., 2., 1., 1., 7., 9.],
        [0., 6., 4., 0., 2., 9., 8., 7.],
        [5., 3., 3., 7., 4., 7., 4., 8.],
        [2., 0., 3., 9., 3., 1., 7., 6.]], dtype=float32),
 array([[6., 1., 2., 3., 5., 3., 3., 8.],
        [8., 5., 0., 0., 6., 7., 5., 3.],
        [2., 7., 5., 6., 6., 3., 5., 1.],
        [4., 4., 4., 0., 0., 4., 7., 3.],
        [3., 4., 1., 1., 4., 8., 9., 2.],
        [6., 7., 1., 2., 6., 8., 4., 7.],
        [5., 8., 3., 8., 3., 3., 1., 9.],
        [2., 4., 2., 9., 6., 2., 9., 6.]], dtype=float32))

In [52]:
%%time 
np.dot(data_A, data_B) #software

CPU times: user 194 µs, sys: 18 µs, total: 212 µs
Wall time: 203 µs


array([[167., 175.,  76., 109., 165., 173., 188., 148.],
       [144., 106.,  44.,  74., 118., 137., 127., 149.],
       [100., 164.,  78., 165., 132., 101., 124., 150.],
       [167., 153.,  80.,  85., 139., 160., 180., 139.],
       [142., 156.,  49., 140., 139., 126., 160., 159.],
       [170., 221.,  69., 171., 188., 180., 175., 203.],
       [178., 198.,  92., 155., 179., 189., 234., 214.],
       [116., 158.,  92., 139., 103., 116., 176., 158.]], dtype=float32)

In [53]:
from pynq import allocate

In [54]:
data_a_f = data_A.flatten()
data_b_f = data_B.flatten()

data_input = np.concatenate([data_a_f, data_b_f])

In [55]:
data_input

array([4., 8., 6., 4., 4., 3., 3., 3., 9., 5., 0., 1., 7., 0., 5., 0., 2.,
       0., 7., 0., 2., 3., 8., 5., 6., 8., 5., 7., 4., 0., 3., 1., 0., 9.,
       0., 2., 1., 1., 7., 9., 0., 6., 4., 0., 2., 9., 8., 7., 5., 3., 3.,
       7., 4., 7., 4., 8., 2., 0., 3., 9., 3., 1., 7., 6., 6., 1., 2., 3.,
       5., 3., 3., 8., 8., 5., 0., 0., 6., 7., 5., 3., 2., 7., 5., 6., 6.,
       3., 5., 1., 4., 4., 4., 0., 0., 4., 7., 3., 3., 4., 1., 1., 4., 8.,
       9., 2., 6., 7., 1., 2., 6., 8., 4., 7., 5., 8., 3., 8., 3., 3., 1.,
       9., 2., 4., 2., 9., 6., 2., 9., 6.], dtype=float32)

In [56]:
input_buffer = allocate(128, np.float32)
output_buffer = allocate((8,8), np.float32)

In [57]:
np.copyto(input_buffer, data_input)

In [58]:
%%time 
data_send.transfer(input_buffer)
data_recv.transfer(output_buffer)
data_send.wait()
data_recv.wait()

CPU times: user 1.09 ms, sys: 103 µs, total: 1.19 ms
Wall time: 927 µs


In [59]:
output_buffer

PynqBuffer([[167., 175.,  76., 109., 165., 173., 188., 148.],
            [144., 106.,  44.,  74., 118., 137., 127., 149.],
            [100., 164.,  78., 165., 132., 101., 124., 150.],
            [167., 153.,  80.,  85., 139., 160., 180., 139.],
            [142., 156.,  49., 140., 139., 126., 160., 159.],
            [170., 221.,  69., 171., 188., 180., 175., 203.],
            [178., 198.,  92., 155., 179., 189., 234., 214.],
            [116., 158.,  92., 139., 103., 116., 176., 158.]],
           dtype=float32)

In [60]:
np.dot(data_A, data_B) - output_buffer

PynqBuffer([[0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)